### We will now use the model to make a prediction on our historical silver table

In [0]:
# create the widget parameters 
dbutils.widgets.text("usernameWidget","","Enter Your Name Here: ")
dbutils.widgets.text("environmentWidget","","Enter Environment: ")

In [0]:
# save as variables 
usernameWidget = dbutils.widgets.get("usernameWidget")
environmentWidget = dbutils.widgets.get("environmentWidget")

assert usernameWidget != "", "Please provide a value for usernameWidget"
assert environmentWidget != "", "Please provide a value for environmentWidget"

print(usernameWidget)
print(environmentWidget)

tau_lim
dev

# Get model uri from the results of Auto-ML
1. Pick the best performing model in your Auto-ML results
1. Click into the experiment
1. Scroll to the bottom to 'Artifacts' and look for 'Full Path'
1. Copy the path and paste it into 'model_uri path below'

In [0]:
import mlflow

model_uri = 'dbfs:/databricks/mlflow-tracking/2923853903717027/76861a3496e346178ca94297af00fcfd/artifacts/model'

# If you've registered the model, you can also easily refer to it like this (change the model name accordingly)
# model_uri = f'models:/tau-lim-churn-prediction/1'

# If you've promoted it to Staging / Production, you can also easily refer to it like this (change the model name accordingly)
# model_uri = f'models:/tau-lim-churn-prediction/Staging'

model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

--------------------------------------------------------------------------- 
 RestException Traceback (most recent call last)
 <command-2467854589205260> in <module> 
 9 # model_uri = f'models:/tau-lim-churn-prediction/Staging' 
 10 
 ---> 11 model = mlflow . pyfunc . spark_udf ( spark , model_uri = model_uri ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/pyfunc/__init__.py in spark_udf (spark, model_uri, result_type) 
 863 
 864 with TempDir ( ) as local_tmpdir : 
 --> 865 local_model_path = _download_artifact_from_uri(
 866 artifact_uri = model_uri , output_path = local_tmpdir . path ( ) 
 867 )

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/artifact_utils.py in _download_artifact_from_uri (artifact_uri, output_path) 
 93 root_uri = prefix + urllib . parse . urlunparse ( parsed_uri ) 
 94 
 ---> 95 return get_artifact_repository(artifact_uri=root_uri).download_artifacts(
 96 artifact_path = artifact_path , dst_path = output_path
 97 )

 /databricks/python/lib/python3.8/site-packages/mlflow/store/artifact/artifact_repository_registry.py in get_artifact_repository (artifact_uri) 
 105 requirements . 
 106 """
 --> 107 return _artifact_repository_registry . get_artifact_repository ( artifact_uri ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/artifact/artifact_repository_registry.py in get_artifact_repository (self, artifact_uri) 
 71 )
 72 )
 ---> 73 return repository ( artifact_uri ) 
 74 
 75 

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/store/entrypoint.py in dbfs_artifact_repo_factory (artifact_uri) 
 77 db_profile_uri = get_databricks_profile_uri_from_artifact_uri ( cleaned_artifact_uri ) 
 78 if is_databricks_acled_artifacts_uri ( artifact_uri ) : 
 ---> 79 return DatabricksArtifactRepository ( cleaned_artifact_uri ) 
 80 elif (
 81 is_dbfs_fuse_available ( ) 

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/store/artifact_repo.py in __init__ (self, artifact_uri) 
 120 # repository will be performed relative to this computed location 
 121 artifact_repo_root_path = extract_and_normalize_path ( artifact_uri ) 
 --> 122 run_artifact_root_uri = self . _get_run_artifact_root ( self . run_id ) 
 123 run_artifact_root_path = extract_and_normalize_path ( run_artifact_root_uri ) 
 124 run_relative_root_path = posixpath.relpath(

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/store/artifact_repo.py in _get_run_artifact_root (self, run_id) 
 162 def _get_run_artifact_root ( self , run_id ) : 
 163 json_body = message_to_json ( GetRun ( run_id = run_id ) ) 
 --> 164 run_response = self . _call_endpoint ( MlflowService , GetRun , json_body ) 
 165 return run_response . run . info . artifact_uri
 166 

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/store/artifact_repo.py in _call_endpoint (self, service, api, json_body) 
 158 endpoint , method = _SERVICE_AND_METHOD_TO_INFO [ service ] [ api ] 
 159 response_proto = api . Response ( ) 
 --> 160 return call_endpoint ( db_creds , endpoint , method , json_body , response_proto ) 
 161 
 162 def _get_run_artifact_root ( self , run_id ) : 

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/utils/rest_utils.py in call_endpoint (host_creds, endpoint, method, json_body, response_proto) 
 287 host_creds = host_creds , endpoint = endpoint , method = method , json = json_body
 288 )
 --> 289 response = verify_rest_response ( response , endpoint ) 
 290 js_dict = json . loads ( response . text ) 
 291 parse_dict ( js_dict = js_dict , message = response_proto ) 

 /databricks/python/lib/python3.8/site-packages/mlflow_databricks_artifacts/utils/rest_utils.py in verify_rest_response (response, endpoint) 
 207 if response . status_code != 200 : 
 208 if _can_parse_as_json_object ( response . text ) : 
 --> 209 raise RestException ( json . loads ( response . text ) ) 
 210 else : 
 211 base_msg = "API request to e

We can check the expected input columns for the model - this is useful when your input dataframe may contain more columns

In [0]:
model_features = model.metadata.get_input_schema().input_names()
print(model_features)

In [0]:
feature_df = spark.sql('select * from ml_workshop.customer_churn_silver')

# Run prediction using ML model

In [0]:
predictions = feature_df.withColumn('churnPredictions', model(*model_features))
display(predictions.select("customerId", "churnPredictions", "churn"))

# Persist predicted churn table to a gold table 
### Gold table can then be used to build a churn prediction dashboard

In [0]:
gold_table_name = 'customer_churn_predictions'

# Drop table if it exists
spark.sql(f"DROP TABLE IF EXISTS {environmentWidget}_{usernameWidget}_db.{gold_table_name}")

In [0]:
predictions.write.mode('overwrite') \
                .saveAsTable(f'{environmentWidget}_{usernameWidget}_db.{gold_table_name}')